In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
#import numpy as np
import json
import time
import numpy as np

reviews = []

def trustpilot_scraper(PATH: str, n_pages):
#Lists
  body = []
  heading = []
  rating = []
  location = []
  author = []
  date = []
  #Website Load
  page = "{}?page=".format(PATH)
  for page_number in range(1, n_pages+1):
    url = "{x}{y}".format(x = page, y = page_number)
    req = requests.get(url)
    time.sleep(2)
    soup = BeautifulSoup(req.text, 'html.parser')
    #initial reviews
    reviews_raw = soup.find("script", id = "__NEXT_DATA__").string
    reviews_raw = json.loads(reviews_raw)
    rev = reviews_raw["props"]["pageProps"]["reviews"]
    #get reviews into df
    for i in range(len(rev)):
      instance = rev[i]
      body_ = instance["text"]
      heading_ = instance["title"]
      rating_ = instance["rating"]
      location_ = instance["consumer"]["countryCode"]
      author_ = instance["consumer"]["displayName"]
      date_ = pd.to_datetime(instance["dates"]["publishedDate"]).strftime("%Y-%m-%d")
      #append to the list
      body.append(body_)
      heading.append(heading_)
      rating.append(rating_)
      location.append(location_)
      author.append(author_)
      date.append(date_)
      df = {
      'date' : date,
      'author' : author,
      'text' : body,
      'title' : heading,
      'stars' : rating,
      'location' : location,
      'url': url
      }
      rev_df = pd.DataFrame(df)
      rev_df.sort_values(by = "date", axis = 0, inplace = True, ignore_index = True)
      rev_df.drop_duplicates(subset=["text"],keep= 'first', inplace= True)
      rev_df.reset_index(drop = True, inplace = True)
  return rev_df

#Set number of pages to scrape
pages = np.arange(1, 100, 1)

web = "vestiairecollective"
#Create a loop to go over the reviews
for page in pages:
  review = trustpilot_scraper("https://www.trustpilot.com/review/"+web+".com?stars=1&stars=2&stars=3", page)
  review.to_json(r"data/{web}/reviews_{page}.json".format(page = page, web = web), orient = 'records')

